In [23]:
from officelib.xllib import *
from officelib import const as c
from hello.hello import *
import os
os.chdir("C:/.replcache")

In [24]:
def download(ip, name, fp):
    app = open_hello(ip)
    app.login()
    r = app.getdatareport_bybatchname(name)
    with open(fp, 'wb') as f:
        f.write(r)

In [25]:
def find_column(rng, header):
    return rng.Find(header, SearchOrder=c.xlByColumns)

def fudge_column(cells):
    """ For some reason, cells.Find() can't find 
    the requested header if the header occurs in the 
    first column, so insert a dummy here """
    cells.Columns(1).Insert()

def get_header_data(cells, h):
    col = find_column(cells, h)
    s = col.Offset(2,2)
    e = s.End(c.xlDown)
    rows = e.Row-s.Row + 1
    return col, s, e, rows

def make_col(ref, offset, header):
    c = ref.Offset(1, offset)
    c.Value = header
    s = c.Offset(2, 1)
    return c, s

def create_data(cells):
    rng=cells.Rows(1)
    fo = get_data("FilterOvenPV(C)")
    ta = get_data("TempA(C)")
    tb = get_data("TempB(C)")
    
    amb = tb.Offset(1,4)
    diff = amb.Offset(1,2)
    
    amb.Value = "Ambient"
    diff.Value = "Diff A-B"
    
    return fo, ta, tb, amb, diff

def make_amb_diff(cells):
    fo, fo_start, fo_end, fo_r = get_header_data(cells, "FilterOvenPV(C)")
    tempa, tempa_start, tempa_end, tempa_r = get_header_data(cells, "TempA(C)")
    tempb, tempb_start, tempb_end, tempb_r = get_header_data(cells, "TempB(C)")

    assert (fo_r == tempa_r and fo_r == tempb_r)
    
    amb, amb_start = make_col(tempb, 4, "Ambient")
    diff, diff_start = make_col(amb, 2, "Diff A-B")

    amb_form = "=%s" % addr(fo_start)
    diff_form = "=%s-%s" % (addr(tempa_start), addr(tempb_start))

    amb_start.Value = amb_form
    diff_start.Value = diff_form
    
    form_rng = cells.Range(amb_start, diff_start)
    dst = cells.Range(amb_start, diff_start.Offset(fo_r, 1))
    form_rng.AutoFill(dst)
    
    return amb_start, diff_start

def addr(r):
    return r.GetAddress(False, False)

def chart_data_rng(cells, xc, yc):
    x = cells.Range(xc, xc.End(c.xlDown))
    y = cells.Range(yc, yc.End(c.xlDown))
    return x, y

def min_and_max(cells, x_start, ystart):
    rng = cells.Range(x_start, x_start.End(c.xlDown))
    yrng = cells.Range(ystart, ystart.End(c.xlDown))
    x = [x1[0] for x1 in rng.Value]
    y = [y1[0] for y1 in yrng.Value]
    xmin = min(x)
    xmax = max(x)
    ymin = min(y)
    ymax = max(y)
    return xmin, xmax, ymin, ymax

def shrink_markers(c, sz=2):
    for s in c.SeriesCollection():
        s.MarkerSize=sz

def analyze_rtd_data(pth, title):
    
    fp = os.path.abspath(pth)
    
    # Init ctx
    xl = Excel(visible=False)
    with HiddenXl(xl):
        wb = xl.Workbooks.Open(fp)
        ws = wb.Worksheets(1)
        cells = ws.Cells  
        _analyze_rtd_data_inner(xl, wb, ws, cells, title)

def _analyze_rtd_data_inner(xl, wb, ws, cells, title):
    # data analysis 
    fudge_column(cells)
    amb_start, diff_start = make_amb_diff(cells)
    
    # create chart ambient vs rtd error
    c1 = CreateChart(ws)
    x, y = chart_data_rng(cells, amb_start, diff_start)
    series = CreateDataSeries(c1, x, y)
    
    FormatChart(c1, None, title, "Ambient Temp(C)", "Err A-B", True, False)
    shrink_markers(c1)
    
    xmin, xmax, ymin, ymax = min_and_max(cells, amb_start, diff_start)
    FormatAxesScale(c1, int(xmin)-2, int(xmax)+2, round(ymin-.2, 1), round(ymax+.2, 1))
        
    c1.Location(c.xlLocationAsNewSheet, "AmbRTD")
    c1 = wb.Charts("AmbRTD")
    
    # create chart amb v. time and diff v. time
    
    # time data
    tb = find_column(cells, "TempB")
    ts = tb.Offset(2,1)
    te = ts.End(c.xlDown)
    t = cells.Range(ts, te)
    
    c2 = CreateChart(ws)
    s2 = CreateDataSeries(c2, t, x, "Ambient")
    s3 = CreateDataSeries(c2, t, y, "Diff")
    FormatChart(c2, None, title, "Time", "Temp(C)")
    s3.AxisGroup = 2
    y2ax = c2.Axes(2, c.xlSecondary); y2ax.HasTitle=True; y2ax.AxisTitle.Text = "Diff A-B"
    
    
    xmin, xmax, y1min, y1max = min_and_max(cells, ts, amb_start)
    _, _, y2min, y2max = min_and_max(cells, ts, diff_start)
    y1min = int(y1min-1)
    y1max = int(y1max+1)
    y2min = round(y2min-.2,1)
    y2max = round(y2max+.2,1)
    FormatAxesScale(c2, None, None, y1min, y1max, y2min, y2max)    
    shrink_markers(c2)
    c2.Axes(1).TickLabels.NumberFormat = "[$-409]h:mm AM/PM;@"
    c2.Location(c.xlLocationAsNewSheet, "vs Time")
    return c1, c2

In [26]:
def download_and_analyze(ip, name, title):
    os.chdir("C:/.replcache")
    fp = name+".csv"
    download(ip, name, fp)
    analyze_rtd_data(fp, title)

In [27]:
download_and_analyze('71.189.82.196:6', 'rtdtest', '15L Ambient RTD Effect')

com_error: (-2147352567, 'Exception occurred.', (0, None, None, None, 0, -2147467259), None)

In [ ]:
analyze_rtd_data